In [450]:
using Revise
using Bilevel

using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Plots
using LinearAlgebra
using StaticArrays

In [670]:
urdf = joinpath("..", "..", "urdf", "little_dog", "little_dog2d.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 10.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles);

In [547]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
open(mvis)
# IJuliaCell(mvis)

Could not open browser automatically: ErrorException("failed process: Process(`xdg-open http://127.0.0.1:8703`, ProcessExited(3)) [3]")
Please open the following URL in your browser:
http://127.0.0.1:8703


┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8703
└ @ MeshCat /home/blandry/.julia/packages/MeshCat/WlA0B/src/servers.jl:24
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 778: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'http://127.0.0.1:8703'


In [652]:
N = 8
Δt = 0.025
q0 = [0., -.275, 0., 0., 0., 0., 0., pi/4, pi/4, -pi/4, -pi/4, -pi/2, -pi/2, pi/2, pi/2] 
v0 = zeros(num_velocities(mechanism))

sim_data = get_sim_data_indirect(mechanism,env,Δt)
null_ctrl! = (u,t,x) -> u[:] .= 0.
x0 = MechanismState(mechanism)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
traj_sim = Bilevel.simulate(sim_data,null_ctrl!,x0,N)
setanimation!(mvis, traj_sim[6], traj_sim[1])

In [653]:
set_configuration!(mvis, traj_sim[1][end])

## Indirect Method

In [671]:
q_nom = copy(traj_sim[1][end])
q_min = min.(q_nom, [-5., -.5, -.5*pi, 0., 0., 0., 0., -3.5, -3.5, -3.5, -3.5, -3.1, -3.1, -3.1, -3.1])
q_max = max.(q_nom, [5., -0.1, .5*pi, 0., 0., 0., 0., 2.4, 2.4, 2.4, 2.4, 1., 1., 1., 1.])

x_goal = 0.1
x_goal_net = 0.1

N = 10
Δt = 0.05

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, num_positions(mechanism), x -> vs(x, :q1) - q_nom)
    add_eq!(sim_data, :cv1, num_velocities(mechanism), x -> vs(x, :v1))
    add_ineq!(sim_data, :cqNx, 1, x -> x_goal - vs(x, Symbol("q", N))[1])
    
#     for n = 2:N
#         add_eq!(sim_data, Symbol("cqs1", n), 1, x -> vs(x, Symbol("q", n))[8] - q_nom[8])
#         add_eq!(sim_data, Symbol("cqs2", n), 1, x -> vs(x, Symbol("q", n))[11] - q_nom[11]) 
#         add_eq!(sim_data, Symbol("cqs3", n), 1, x -> vs(x, Symbol("q", n))[12] - q_nom[12]) 
#         add_eq!(sim_data, Symbol("cqs4", n), 1, x -> vs(x, Symbol("q", n))[15] - q_nom[15]) 
#     end
end

function add_prob_obj!(sim_data)
    for n = 1:N-1
        add_obj!(sim_data, Symbol("u", n), x -> sim_data.vs(x, Symbol("u", n))' * sim_data.vs(x, Symbol("u", n)))
    end
end

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [1. * Δt], [1. * Δt], 1:N-1)
    
    # torques limits
    u_limit = 10.
    add_box_con_snopt!(x_min, x_max, sim_data, :u, -u_limit * ones(num_velocities(mechanism)), u_limit * ones(num_velocities(mechanism)), 1:N-1)
    
    x_min, x_max
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
        qn = copy(q_nom)
        qn[1] = (n/N)*(x_goal - q_nom[1]) + q_nom[1]
        x0[sim_data.vs(Symbol("q", n))] .= qn
    end
    
    x0
end

function setup_prob!(sim_data)
    add_prob_constraints!(sim_data)
    add_prob_obj!(sim_data)
    x_min, x_max = get_prob_limits(sim_data)
    x0 = get_prob_init(sim_data)
    
    x0, x_min, x_max
end

setup_prob! (generic function with 1 method)

In [672]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data);

In [673]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 10000

traj_indirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter);

x0 = traj_indirect[9]

setanimation!(mvis, traj_indirect[7], traj_indirect[1])

Resource limit error: iteration limit reached


In [559]:
q_mid = traj_indirect[1][end];
v_mid = traj_indirect[2][end];

In [560]:
function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, num_positions(mechanism), x -> vs(x, :q1) - q_mid)
    add_eq!(sim_data, :cv1, num_velocities(mechanism), x -> vs(x, :v1) - v_mid)
    add_ineq!(sim_data, :cqNx, 1, x -> x_goal_net - vs(x, Symbol("q", N))[1])
    add_ineq!(sim_data, :cqN, num_positions(mechanism)-1, x -> vs(x, Symbol("q", N))[2:end] - q_nom[2:end])
    add_eq!(sim_data, :cvN, num_velocities(mechanism), x -> vs(x, Symbol("v", N)))
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
        qn = copy(q_nom)
        qn[1] = x_goal
        x0[sim_data.vs(Symbol("q", n))] .= qn
    end
    
    x0
end

get_prob_init (generic function with 1 method)

In [561]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data);

In [568]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 30000

traj_indirect2 = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter);

x0 = traj_indirect2[9]

setanimation!(mvis, traj_indirect2[7], traj_indirect2[1])

Resource limit error: iteration limit reached


In [523]:
t_step = vcat(traj_indirect[7], traj_indirect2[7] .+ traj_indirect[7][end])
q_step = vcat(traj_indirect[1], traj_indirect2[1]);

In [525]:
reps = 1

t = t_step
qtraj = q_step
for i = 1:(reps-1)
    t = vcat(t, t_step .+ t[end])
    qtraj = vcat(qtraj, map(q -> q + vcat(qtraj[end][1],zeros(length(q)-1)), q_step))
end

setanimation!(mvis, t, qtraj)

## Semidirect Method

In [545]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data);

In [ ]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 5000
steps = 12

for n = 1:N-1
    sim_data.fric_options[n]["num_sosteps"] = steps
end

traj_semidirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter)

x0 = traj_semidirect[9]

setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])

In [557]:
setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])

In [ ]:
q_mid = traj_semidirect[1][end]

In [ ]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 50000
steps = 10

for n = 1:N-1
    sim_data.fric_options[n]["num_sosteps"] = steps
end

traj_semidirect2 = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter)

x0 = traj_semidirect2[9]

setanimation!(mvis, traj_semidirect2[7], traj_semidirect2[1])

In [ ]:
t_step = vcat(traj_semidirect[7], traj_semidirect2[7] .+ traj_semidirect[7][end])
q_step = vcat(traj_semidirect[1], traj_semidirect2[1]);